# Import Module

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd

# Load Data

In [2]:
temps = pd.read_csv("./weather.csv")
temps.head()

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森


# Scraping

In [12]:
# dfs = pd.DataFrame()

In [232]:
path = "/home/higuchi-lab/anaconda3/lib/python3.8/site-packages/chromedriver_binary/chromedriver"
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2022&month=05&day=&view=p1s"
# TODO: should change URL automatically correspoding to temps["area"].unique()

browser = webdriver.Chrome(executable_path=path)
browser.get(url)

In [223]:
target = temps["area"].unique()[31]
print(target)

東京


In [224]:
df = pd.DataFrame()

for i in range(31):
    where = 5 + i
    texts = browser.find_element_by_xpath(f"/html/body/div[2]/div/div[2]/div[2]/div[4]/table/tbody/tr[{where}]")
    texts = texts.text.split(" ")
    texts = [text for text in texts if text != ")"]
    row = {
        "date": f"2022-05-{i+1}",
        "mean_temp": texts[5],
        "max_temp": texts[6],
        'max_temp_time': 0, 
        "min_temp": texts[7],
        "min_temp_time": 0,
        "sum_rain": texts[2],
        "sun_time": texts[15],
        "mean_humid": texts[8],
        "area": target
    }
    df = pd.concat([df, pd.DataFrame(row, index=[i])], axis=0)

In [225]:
dfs = pd.concat([dfs, df], axis=0)

In [226]:
dfs

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,2022-05-1,10.1,13.8,0,6.2,0,0.0,0.0,66,青森
1,2022-05-2,9.8,15.0,0,5.4,0,0.0,9.1,62,青森
2,2022-05-3,10.9,16.1,0,6.2,0,3.0,5.4,62,青森
3,2022-05-4,14.3,20.1,0,9.5,0,0.0,8.8,60,青森
4,2022-05-5,12.6,17.8,0,7.1,0,--,12.0,74,青森
...,...,...,...,...,...,...,...,...,...,...
26,2022-05-27,21.0,24.6,0,19.2,0,59.5,1.2,91,東京
27,2022-05-28,22.4,27.7,0,16.8,0,--,10.1,62,東京
28,2022-05-29,24.1,31.2,0,16.6,0,--,13.3,53,東京
29,2022-05-30,22.3,27.8,0,18.8,0,--,11.4,70,東京


In [227]:
# dfs.to_csv("./tmp_scraped_weather.csv", index=False)

# Concatenation Between Train Weather And Test Weather

In [229]:
temps = pd.read_csv("./weather.csv")
temps["date"] = pd.to_datetime(temps["date"], format="%Y%m%d")

test_temps = pd.read_csv("./tmp_scraped_weather.csv")
test_temps["date"] = pd.to_datetime(test_temps["date"], format="%Y-%m-%d")

temps = pd.concat([temps, test_temps], axis=0)
# temps.to_csv("./scraped_weather.csv", index=False)